In [95]:
import pandas as pd

from nebula3.Config import Config
from nebula3.gclient.net import ConnectionPool

# Load data

In [96]:
df = pd.read_csv('data_test.csv', sep=';')

In [97]:
# get array of unique users
users = list(set(list(df['ФИО участника события 1']) + list(df['ФИО участника события 2'])))
unique_users = {users[i]: i for i in range(len(users))}
len(unique_users)

9899

# About DB

### In this data base I created some objects in nebula-console:

SPACE `network`

TAG `user(name string)`

EDGE `request(id int)`

### Main idea users pull requests to another users.

In [98]:
# define a config
config = Config()
config.max_connection_pool_size = 10

# init connection pool
connection_pool = ConnectionPool()

# if the given servers are ok, return true, else return false
connection_pool.init([('127.0.0.1', 9669)], config)

True

In [99]:
# option 1 control the connection release yourself
# get session from the pool
session = connection_pool.get_session('root', 'nebula')
session.execute('use network;');

### Creation process

In [100]:
session.execute('DROP SPACE network;')
session.execute('CREATE SPACE network(partition_num=5, replica_factor=1, vid_type=int64);')
session.execute('USE network;');

In [101]:
session.execute('CREATE TAG user(name string);')
session.execute('CREATE EDGE request(id int);');

In [102]:
# add users

for i in unique_users:
    message = 'INSERT VERTEX user(name) VALUES {}:("{}");'.format(unique_users[i], i)
    session.execute(message);

In [104]:
for i in range(len(df['id события'])):
    message = 'INSERT EDGE request(id) VALUES {} -> {}:({})'.format(unique_users[df['ФИО участника события 1'][i]], 
                                                                    unique_users[df['ФИО участника события 2'][i]], 
                                                                    df['id события'][i])
    session.execute(message);

In [105]:
session.execute('CREATE TAG INDEX IF NOT EXISTS name ON user(name(200));')
session.execute('CREATE EDGE INDEX IF NOT EXISTS request_index on request();');

In [106]:
session.execute('REBUILD TAG INDEX name;')
session.execute('REBUILD EDGE INDEX follow_index;')

### Tests

In [108]:
session.execute('FETCH PROP ON user 1 YIELD properties(vertex).name;')

ResultSet(keys: ['properties(VERTEX).name'], values: ["Петруняк Вероника Денисовна"])

In [10]:
session.execute('match (v:user{name: "Ахромеева Алина Ивановна"})-->(v2) return v2')

ResultSet(keys: ['v2'], values: [(7290 :user{name: "Бордачев Никита Васильевич"})],[(9827 :user{name: "Ящукова Любовь Ефимовна"})],[(2307 :user{name: "Преображенская Кира Альбертовна"})],[(1631 :user{name: "Щенников Дмитрий Григорьевич"})],[(3156 :user{name: "Расулев Никита Петрович"})],[(6473 :user{name: "Чикирева Мария Романовна"})],[(2968 :user{name: "Гужов Глеб Данилович"})],[(3841 :user{name: "Вохменцев Владимир Владиславович"})],[(793 :user{name: "Старухин Дамир Маратович"})],[(5446 :user{name: "Акодес Ефим Анатольевич"})],[(255 :user{name: "Урманцева Евгения Олеговна"})],[(4620 :user{name: "Борголов Евгений Маратович"})],[(7342 :user{name: "Бобрецова Светлана Артемовна"})],[(1342 :user{name: "Толкунова Валентина Маратовна"})],[(7640 :user{name: "Бугаенкова Карина Аркадьевна"})],[(9708 :user{name: "Тяжлов Ринат Владиславович"})],[(776 :user{name: "Шальнова Ольга Владимировна"})],[(7524 :user{name: "Сарсадских Алена Геннадьевна"})],[(55 :user{name: "Камилов Дамир Павлович"})],[(34

In [107]:
session.execute('match (v1:user)-->(v2:user) return v1.user.name as v1, v2.user.name as v2 limit 10;')

ResultSet(keys: ['v1', 'v2'], values: ["Яшник Гульнара Витальевна", "Касько Альбина Маратовна"],["Ясинская Яна Борисовна", "Григоревский Никита Иванович"],["Ярмольник Гульнара Александровна", "Дорошевич Аркадий Робертович"],["Яночка Римма Артуровна", "Савоничев Петр Вадимович"],["Ямнова Татьяна Михаиловна", "Носенкова Жанна Денисовна"],["Юхтин Валентин Ярославович", "Хаджаева Альбина Юрьевна"],["Юхина Анна Владимировна", "Вейцман Вероника Эдуардовна"],["Юхиева Кристина Филипповна", "Иванко Раиса Степановна"],["Юрмегова Мария Руслановна", "Чепик Данил Рамилевич"],["Юринская Елизавета Никитовна", "Башаев Алексей Никитович"])

### Disconnection

In [20]:
# release session
session.release()

# close the pool
connection_pool.close()